In [29]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [30]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [31]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


    col1

In [34]:
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter

    c1 = 2
    c2 = 2
    r1 = [0, 1]
    r2 = [0, 1]
    iteration_max = 40
    particles = 70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    temp = 0
    ae = 0

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        #inisialisasi posisi awal
        particles_position = np.zeros(n_particles)
    
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        particles_position = np.zeros(n_particles)
    
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        particles_position = np.zeros(n_particles)
      
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    swarm_size = 10
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    
    for i in range(swarm_size):    
        xSimple = randomSimpleUCW()
        xAverage = randomAverageUCW()
        xComplex = randomComplexUCW()
        case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
        estimated_effort = case * 20
        
        #Selisih estimasi effort
        ae = abs(estimated_effort - real_effort)
        
        #menjumlah 10 angka selisih
        temp = np.array(temp+ae)
         
        print('Estimasi ke', i+1,':',estimated_effort)
        #print('ucp   ', case)
        #print('ae    ', ae)
        #print('S     ', xSimple)
        #print('A     ', xAverage)
        #print('C     ', xComplex)
        #print('estimated effort', estimated_effort)
    mae = temp/swarm_size
    print('MAE :', mae)
    print()
    


____ DATA  1 ____
Estimasi ke 1 : 4431.66487917109
Estimasi ke 2 : 5071.535602375591
Estimasi ke 3 : 4797.006724915205
Estimasi ke 4 : 5215.943504191137
Estimasi ke 5 : 5204.132351789436
Estimasi ke 6 : 4632.646841459362
Estimasi ke 7 : 4227.866813715582
Estimasi ke 8 : 4805.7260489256
Estimasi ke 9 : 4611.321494969631
Estimasi ke 10 : 5020.135956613189
MAE : 3168.2019781874174


____ DATA  2 ____
Estimasi ke 1 : 8325.91850578222
Estimasi ke 2 : 7702.046602141896
Estimasi ke 3 : 8620.036105022362
Estimasi ke 4 : 8156.034953468641
Estimasi ke 5 : 8722.356808873777
Estimasi ke 6 : 9116.728790451672
Estimasi ke 7 : 8948.338892447899
Estimasi ke 8 : 8151.628599108024
Estimasi ke 9 : 9225.798032028904
Estimasi ke 10 : 8713.239000078167
MAE : 658.2033085119771


____ DATA  3 ____
Estimasi ke 1 : 5866.880970639477
Estimasi ke 2 : 5781.501844783319
Estimasi ke 3 : 5821.6005461080895
Estimasi ke 4 : 5610.08428750314
Estimasi ke 5 : 5569.615555406204
Estimasi ke 6 : 5724.580636590537
Estimasi k

Estimasi ke 4 : 5019.587430274113
Estimasi ke 5 : 4729.218461264141
Estimasi ke 6 : 4714.058400911154
Estimasi ke 7 : 3951.1110490482547
Estimasi ke 8 : 4199.410020055027
Estimasi ke 9 : 4149.338822613543
Estimasi ke 10 : 3898.925816035255
MAE : 2447.209523418561


____ DATA  31 ____
Estimasi ke 1 : 7368.7055928866275
Estimasi ke 2 : 6694.719544804631
Estimasi ke 3 : 6607.665773049004
Estimasi ke 4 : 7466.371623595916
Estimasi ke 5 : 6759.629862744361
Estimasi ke 6 : 6983.459138765955
Estimasi ke 7 : 6477.45791149336
Estimasi ke 8 : 6275.286230549104
Estimasi ke 9 : 5895.361248190795
Estimasi ke 10 : 7127.7895523244615
MAE : 376.4904151839689


____ DATA  32 ____
Estimasi ke 1 : 4842.394517697619
Estimasi ke 2 : 4538.452370012072
Estimasi ke 3 : 4827.890211654117
Estimasi ke 4 : 4652.859443563212
Estimasi ke 5 : 5518.36495962707
Estimasi ke 6 : 5407.342368163663
Estimasi ke 7 : 4694.164847872481
Estimasi ke 8 : 4395.182381888327
Estimasi ke 9 : 5271.627102507259
Estimasi ke 10 : 5088.3

____ DATA  54 ____
Estimasi ke 1 : 7118.423914383079
Estimasi ke 2 : 6293.018955521628
Estimasi ke 3 : 7197.784647821069
Estimasi ke 4 : 7195.048217850961
Estimasi ke 5 : 7061.509132179962
Estimasi ke 6 : 6676.253768701962
Estimasi ke 7 : 6431.175721249002
Estimasi ke 8 : 6888.589610943511
Estimasi ke 9 : 6330.234535419217
Estimasi ke 10 : 6537.380749502103
MAE : 810.9419253572494


____ DATA  55 ____
Estimasi ke 1 : 7744.85227608967
Estimasi ke 2 : 7935.814338464457
Estimasi ke 3 : 7368.562469090524
Estimasi ke 4 : 7277.833974578048
Estimasi ke 5 : 7513.385992414243
Estimasi ke 6 : 6698.392115380757
Estimasi ke 7 : 7917.057971589598
Estimasi ke 8 : 7399.28429623561
Estimasi ke 9 : 7497.148199783983
Estimasi ke 10 : 6949.174971181803
MAE : 1486.1506604808694


____ DATA  56 ____
Estimasi ke 1 : 8948.508860639651
Estimasi ke 2 : 8409.034034078446
Estimasi ke 3 : 9249.421656396471
Estimasi ke 4 : 8810.7992496951
Estimasi ke 5 : 8974.84088415446
Estimasi ke 6 : 9004.090841922742
Estimasi 